# Full MIDI Music Transformer (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (GPU CHECK)

In [ ]:
#@title NVIDIA GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone --depth 1 https://github.com/asigalov61/Full-MIDI-Music-Transformer
!pip install huggingface_hub
!pip install torch
!pip install einops
!pip install torch-summary
!pip install sklearn
!pip install tqdm
!pip install matplotlib
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio

In [ ]:
#@title Import modules

print('=' * 70)
print('Loading core Full MIDI Music Transformer modules...')

import os
import pickle
import secrets
import statistics
from time import time
import tqdm

from huggingface_hub import hf_hub_download

print('=' * 70)
print('Loading main Full MIDI Music Transformer modules...')
import torch

%cd /content/Full-MIDI-Music-Transformer

import TMIDIX
from x_transformer import *

%cd /content/
print('=' * 70)
print('Loading aux Full MIDI Music Transformer modules...')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

from midi2audio import FluidSynth
from IPython.display import Audio, display

import random

from google.colab import files

print('=' * 70)
print('Done!')
print('Enjoy! :)')
print('=' * 70)

# (LOAD MODEL)

# Choose one and do not forget to restart colab runtime if switching between models

In [ ]:
#@title Load Full MIDI Music Transformer Tiny Model (FAST)

#@markdown Very fast model, 16 layers, 225k MIDIs training corpus

full_path_to_model_checkpoint = "/content/Full-MIDI-Music-Transformer/Models/Tiny/Full_MIDI_Music_Transformer_Tiny_Trained_Model_30000_steps_0.2859_loss_0.9167_acc.pth" #@param {type:"string"}

#@markdown Model precision option

model_precision = "bfloat16-float16" # @param ["bfloat16-float16", "float32"]

#@markdown bfloat16-float16 == Half precision/double speed

#@markdown float32 == Full precision/normal speed

plot_tokens_embeddings = False # @param {type:"boolean"}

print('=' * 70)
print('Loading Full MIDI Music Transformer Tiny Pre-Trained Model...')
print('Please wait...')
print('=' * 70)

if os.path.isfile(full_path_to_model_checkpoint):
  print('Model already exists...')

else:
  hf_hub_download(repo_id='asigalov61/Full-MIDI-Music-Transformer',
                  filename='Full_MIDI_Music_Transformer_Tiny_Trained_Model_30000_steps_0.2859_loss_0.9167_acc.pth',
                  local_dir='/content/Full-MIDI-Music-Transformer/Models/Tiny/',
                  local_dir_use_symlinks=False)
print('=' * 70)
print('Instantiating model...')

torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda'

if model_precision == 'bfloat16-float16':
  dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
else:
  dtype = 'float32'

ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

SEQ_LEN = 4096

# instantiate the model

model = TransformerWrapper(
    num_tokens = 1564,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 1024, depth = 18, heads = 8)
)

model = AutoregressiveWrapper(model, ignore_index = 1563)

model = torch.nn.DataParallel(model)

model.cuda()
print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(full_path_to_model_checkpoint))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)
print('Model will use', dtype, 'precision')
print('=' * 70)

# Model stats
print('Model summary...')
summary(model)

# Plot Token Embeddings

if plot_tokens_embeddings:

  tok_emb = model.module.net.token_emb.emb.weight.detach().cpu().tolist()

  cos_sim = metrics.pairwise_distances(
    tok_emb, metric='cosine'
  )
  plt.figure(figsize=(7, 7))
  plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
  im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
  plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
  plt.xlabel("Position")
  plt.ylabel("Position")
  plt.tight_layout()
  plt.plot()
  plt.savefig("/content/Full-MIDI-Music-Transformer-Tiny-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# (GENERATE)

# (IMPROV)

In [ ]:
#@title Standard Improv Generator

#@markdown Improv settings

#@markdown If first note MIDI patch number == -1 the model will generate first note itself

first_note_MIDI_patch_number = 0 # @param {type:"slider", min:-1, max:128, step:1}

#@markdown Generation settings

number_of_tokens_tp_generate = 700 # @param {type:"slider", min:30, max:4095, step:5}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}
temperature = 0.9 #@param {type:"slider", min:0.1, max:1, step:0.1}

#@markdown Other settings

render_MIDI_to_audio = True # @param {type:"boolean"}

print('=' * 70)
print('Full MIDI Music Transformer Improv Model Generator')
print('=' * 70)

outy = [1562, 1562, 1562, 1562, 1562]

if first_note_MIDI_patch_number > -1:
  outy.extend([first_note_MIDI_patch_number, 0+256, 16+256+128, 72+256+128+128, 90+256+128+128+256])

print('Selected Improv Sequence:')
print(outy[:10])
print('=' * 70)

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

with ctx:
  out = model.module.generate(inp,
                        number_of_tokens_tp_generate,
                        temperature=temperature,
                        return_prime=True,
                        verbose=True)

out0 = out.tolist()

print('=' * 70)
print('Done!')
print('=' * 70)

#======================================================================

print('Rendering results...')

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:10])
  print('=' * 70)

  if len(out1) != 0:

      song = out1
      song_f = []

      time = 0
      dur = 0
      vel = 90
      pitch = 0
      channel = 0

      son = []
      song1 = []
      for j in range(0, len(song), 5): # creating penta seqs...
          song1.append(song[j:j+5])

      patch_list = [0] * 16
      patch_list[9] = 128

      channels_list = [0] * 16
      channels_list[9] = 1

      for s in song1: # decoding...

          # 1553 - pad token

          # 1554 - patch change token
          # 1555 - control change token
          # 1556 - key after touch token
          # 1557 - channel after touch token
          # 1558 - pitch wheel change token
          # 1559 - counters seq token

          # 1560 - outro token
          # 1561 - end token
          # 1562 - start token

          if s[0] < 256: # Note

              patch = s[0]
              time += (s[1]-256) * 16
              dur = (s[2]-256-128) * 32
              pitch = (s[3]-256-128-128) % 128
              vel = (s[4]-256-128-128-256)

              if patch in patch_list:
                  channel = patch_list.index(patch)
                  channels_list[channel] = 1

              else:
                  if 0 in channels_list:
                    channel = channels_list.index(0)
                    channels_list[channel] = 1
                    song_f.append(['patch_change', time, channel, patch])

                  else:
                    channel = 15
                    channels_list[channel] = 1
                    song_f.append(['patch_change', time, channel, patch])

              song_f.append(['note', time, dur, channel, pitch, vel])

          if s[0] == 1554: # patch change

              time += (s[1]-256) * 16
              channel = (s[2]-(256+128+128+256+128))
              patch = s[3]

              if channel != 9:
                  patch_list[channel] = patch
              else:
                  patch_list[channel] = patch + 128

              song_f.append(['patch_change', time, channel, patch])

          if s[0] == 1555: # control change

              time += (s[1]-256) * 16
              patch = s[2]
              controller = (s[3]-(256+128+128+256+128+16))
              controller_value = (s[4]-(256+128+128+256+128+16+128))

              try:
                  channel = patch_list.index(patch)
              except:
                  channel = 15

              song_f.append(['control_change', time, channel, controller, controller_value])

          if s[0] == 1556: # key after touch

              time += (s[1]-256) * 16
              patch = s[2]
              pitch = (s[3]-256-128-128) % 128
              vel = (s[4]-256-128-128-256)

              try:
                  channel = patch_list.index(patch)
              except:
                  channel = 15

              song_f.append(['key_after_touch', time, channel, pitch, vel])

          if s[0] == 1557: # channel after touch

              time += (s[1]-256) * 16
              patch = s[2]
              vel = (s[3]-256-128-128-256)

              try:
                  channel = patch_list.index(patch)
              except:
                  channel = 15

              song_f.append(['channel_after_touch', time, channel, vel])

          if s[0] == 1558: # pitch wheel change

              time += (s[1]-256) * 16
              patch = s[2]
              pitch_wheel = (s[3]-(256+128+128+256+128+16+128)) * 128

              try:
                  channel = patch_list.index(patch)
              except:
                  channel = 15

              song_f.append(['pitch_wheel_change', time, channel, pitch_wheel])

      detailed_stats = TMIDIX.Tegridy_SONG_to_Full_MIDI_Converter(song_f,
                                                                  output_signature = 'Full MIDI Music Transformer',
                                                                  output_file_name = '/content/Full-MIDI-Music-Transformer-Composition_'+str(i),
                                                                  track_name='Project Los Angeles'
                                                                  )

      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Full-MIDI-Music-Transformer-Composition_'+str(i)

      x = []
      y =[]
      c = []

      colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver', 'red', 'yellow', 'green', 'cyan']

      for s in song_f:
        if s[0] == 'note':
          x.append(s[1] / 1000)
          y.append(s[4])
          c.append(colors[s[3]])

      if render_MIDI_to_audio:
        FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
        display(Audio(str(fname + '.wav'), rate=16000))

      plt.figure(figsize=(14,5))
      ax=plt.axes(title=fname)
      ax.set_facecolor('black')

      plt.scatter(x,y, c=c)
      plt.xlabel("Time")
      plt.ylabel("Pitch")
      plt.show()

# (CUSTOM MIDI)

In [ ]:
#@title Load Seed MIDI
select_seed_MIDI = "Upload your own custom MIDI" # @param ["Upload your own custom MIDI", "Desperado Por Amor", "Honesty", "House Of The Rising Sun", "Nothing Else Matters", "Sharing The Night Together"]
render_MIDI_to_audio = False # @param {type:"boolean"}

print('=' * 70)
print('Full MIDI Music Transformer Seed MIDI Loader')
print('=' * 70)


f = ''

if select_seed_MIDI != "Upload your own custom MIDI":
  print('Loading seed MIDI...')
  f = '/content/Full-MIDI-Music-Transformer/Seeds/'+select_seed_MIDI+'.mid'
  ms_score = TMIDIX.midi2single_track_ms_score(open(f, 'rb').read(), recalculate_channels=False)

else:
  print('Upload your own custom MIDI from your computer...')
  print('=' * 70)
  uploaded_MIDI = files.upload()

  if list(uploaded_MIDI.keys()):
    f = list(uploaded_MIDI.keys())[0]
    ms_score = TMIDIX.midi2single_track_ms_score(list(uploaded_MIDI.values())[0], recalculate_channels=False)


if f:
  print('=' * 70)
  print('File:', f)
  print('=' * 70)

  #=======================================================
  # START PROCESSING

  events_matrix1 = []

  itrack = 1

  events_types = ['note',
                  'patch_change',
                  'control_change',
                  'key_after_touch',
                  'channel_after_touch',
                  'pitch_wheel_change']

  while itrack < len(ms_score):
      for event in ms_score[itrack]:
          if event[0] in events_types:
              events_matrix1.append(event)
      itrack += 1

  events_matrix1.sort(key = lambda x: (x[4] if x[0] == 'note' else x[1]), reverse = True)
  events_matrix1.sort(key = lambda x: x[1])

  if len(events_matrix1) > 0:
      if min([e[1] for e in events_matrix1]) >= 0 and min([e[2] for e in events_matrix1 if e[0] == 'note']) >= 0:

          #=======================================================
          # PRE-PROCESSING

          # recalculating timings
          for e in events_matrix1:
              e[1] = int(e[1] / 16) # Max 2 seconds for start-times
              if e[0] == 'note':
                  e[2] = int(e[2] / 32) # Max 4 seconds for durations

          #=======================================================
          # FINAL PRE-PROCESSING

          patch_list = [0] * 16
          patch_list[9] = 128

          melody_chords = []

          pe = events_matrix1[0]

          for e in events_matrix1:

              if e[0] == 'note':

                  # Cliping all values...
                  time = max(0, min(127, e[1]-pe[1]))
                  dur = max(1, min(127, e[2]))
                  cha = max(0, min(15, e[3]))
                  ptc = max(1, min(127, e[4]))
                  vel = max(1, min(127, e[5]))
                  pat = patch_list[cha]

                  # Writing final note
                  melody_chords.append(['note', time, dur, cha, ptc, vel, pat])

              if e[0] == 'patch_change':

                  # Cliping all values...
                  time = max(0, min(127, e[1]-pe[1]))
                  cha = max(0, min(15, e[2]))
                  ptc = max(0, min(127, e[3]))

                  if cha != 9:
                      patch_list[cha] = ptc
                  else:
                      patch_list[cha] = ptc+128

                  melody_chords.append(['patch_change', time, cha, ptc])

              if e[0] == 'control_change':

                  # Cliping all values...
                  time = max(0, min(127, e[1]-pe[1]))
                  cha = max(0, min(15, e[2]))
                  con = max(0, min(127, e[3]))
                  cval = max(0, min(127, e[4]))

                  pat = patch_list[cha]

                  melody_chords.append(['control_change', time, pat, con, cval])

              if e[0] == 'key_after_touch':

                  # Cliping all values...
                  time = max(0, min(127, e[1]-pe[1]))
                  cha = max(0, min(15, e[2]))
                  ptc = max(1, min(127, e[3]))
                  vel = max(1, min(127, e[4]))

                  pat = patch_list[cha]

                  melody_chords.append(['key_after_touch', time, pat, ptc, vel])

              if e[0] == 'channel_after_touch':

                  # Cliping all values...
                  time = max(0, min(127, e[1]-pe[1]))
                  cha = max(0, min(15, e[2]))
                  vel = max(1, min(127, e[3]))

                  pat = patch_list[cha]

                  melody_chords.append(['channel_after_touch', time, pat, vel])

              if e[0] == 'pitch_wheel_change':

                  # Cliping all values...
                  time = max(0, min(127, e[1]-pe[1]))
                  cha = max(0, min(15, e[2]))
                  wheel = max(-8192, min(8192, e[3])) // 128

                  pat = patch_list[cha]

                  melody_chords.append(['pitch_wheel_change', time, pat, wheel])

              pe = e


          #=======================================================

          # Adding SOS/EOS, intro and counters

          if len(melody_chords) < (127 * 100) and ((events_matrix1[-1][1] * 16) < (8 * 60 * 1000)): # max 12700 MIDI events and max 8 min per composition

              melody_chords1 = [['start', 0, 0, 0, 0, 0]]

              events_block_counter = 0
              time_counter = 0

              for i in range(len(melody_chords)):
                  melody_chords1.append(melody_chords[i])

                  time_counter += melody_chords[i][1]

                  if i != 0 and (len(melody_chords) - i == 100):
                      melody_chords1.append(['outro', 0, 0, 0, 0, 0])

                  if i != 0 and (i % 100 == 0) and (len(melody_chords) - i >= 100):
                      melody_chords1.append(['counters_seq', ((time_counter * 16) // 3968), events_block_counter, 0, 0, 0])
                      events_block_counter += 1

              melody_chords1.append(['end', 0, 0, 0, 0, 0])

              #=======================================================

              melody_chords2 = []

              for m in melody_chords1:

                  if m[0] == 'note':

                      if m[3] == 9:
                          ptc = m[4] + 128
                      else:
                          ptc = m[4]

                      # Writing final note
                      melody_chords2.extend([m[6], m[1]+256, m[2]+256+128, ptc+256+128+128, m[5]+256+128+128+256])

                  # Total tokens so far 896

                  if m[0] == 'patch_change': # 896

                      melody_chords2.extend([1554, m[1]+256, m[2]+256+128+128+256+128, m[3], 1553])

                  # Total tokens so far 912

                  if m[0] == 'control_change': # 912

                      melody_chords2.extend([1555, m[1]+256, m[2], m[3]+256+128+128+256+128+16, m[4]+256+128+128+256+128+16+128])

                  # Total tokens so far 1168

                  if m[0] == 'key_after_touch': # 1168

                      if m[2] == 9:
                          ptc = m[3] + 128
                      else:
                          ptc = m[3]

                      melody_chords2.extend([1556, m[1]+256, m[2], ptc+256+128+128, m[4]+256+128+128+256])

                  # Total tokens so far 1168

                  if m[0] == 'channel_after_touch': # 1168

                      melody_chords2.extend([1557, m[1]+256, m[2], m[3]+256+128+128+256, 1553])

                  # Total tokens so far 1168

                  if m[0] == 'pitch_wheel_change': # 1168

                      melody_chords2.extend([1558, m[1]+256, m[2], m[3]+256+128+128+256+128+16+128, 1553])

                  # Total tokens so far 1296

                  if m[0] == 'counters_seq': # 1296

                      melody_chords2.extend([1559, m[1]+256+128+128+256+128+16+128+128, m[2]+256+128+128+256+128+16+128+128+128, 1553, 1553])

                  # Total tokens so far: 1552

                  #=======================================================

                  # 1553 - pad token

                  # 1554 - patch change token
                  # 1555 - control change token
                  # 1556 - key after touch token
                  # 1557 - channel after touch token
                  # 1558 - pitch wheel change token
                  # 1559 - counters seq token

                  # 1560 - outro token
                  # 1561 - end token
                  # 1562 - start token

                  # if m[0] == 'outro':
                  #    melody_chords2.extend([1560, 1560, 1560, 1560, 1560])

                  # if m[0] == 'end':
                  #    melody_chords2.extend([1561, 1561, 1561, 1561, 1561])

                  if m[0] == 'start':
                      melody_chords2.extend([1562, 1562, 1562, 1562, 1562])

  #=======================================================

  # FINAL TOTAL TOKENS: 1562

  #=======================================================

  melody_chords_f = []

  melody_chords_f.extend(melody_chords2)

  print('Composition stats:')
  print('Composition has', len(melody_chords_f), 'tokens')
  print('=' * 70)

  song = melody_chords_f

  song_f = []

  time = 0
  dur = 0
  vel = 90
  pitch = 0
  channel = 0

  son = []
  song1 = []
  for j in range(0, len(song), 5): # creating penta seqs...
      song1.append(song[j:j+5])

  patch_list = [0] * 16
  patch_list[9] = 128

  channels_list = [0] * 16
  channels_list[9] = 1

  for s in song1: # decoding...

      # 1553 - pad token

      # 1554 - patch change token
      # 1555 - control change token
      # 1556 - key after touch token
      # 1557 - channel after touch token
      # 1558 - pitch wheel change token
      # 1559 - counters seq token

      # 1560 - outro token
      # 1561 - end token
      # 1562 - start token

      if s[0] < 256: # Note

          patch = s[0]
          time += (s[1]-256) * 16
          dur = (s[2]-256-128) * 32
          pitch = (s[3]-256-128-128) % 128
          vel = (s[4]-256-128-128-256)

          if patch in patch_list:
              channel = patch_list.index(patch)
              channels_list[channel] = 1

          else:
              if 0 in channels_list:
                channel = channels_list.index(0)
                channels_list[channel] = 1
                song_f.append(['patch_change', time, channel, patch])

              else:
                channel = 15
                channels_list[channel] = 1
                song_f.append(['patch_change', time, channel, patch])

          song_f.append(['note', time, dur, channel, pitch, vel])

      if s[0] == 1554: # patch change

          time += (s[1]-256) * 16
          channel = (s[2]-(256+128+128+256+128))
          patch = s[3]

          if channel != 9:
              patch_list[channel] = patch
          else:
              patch_list[channel] = patch + 128

          song_f.append(['patch_change', time, channel, patch])

      if s[0] == 1555: # control change

          time += (s[1]-256) * 16
          patch = s[2]
          controller = (s[3]-(256+128+128+256+128+16))
          controller_value = (s[4]-(256+128+128+256+128+16+128))

          try:
              channel = patch_list.index(patch)
          except:
              channel = 15

          song_f.append(['control_change', time, channel, controller, controller_value])

      if s[0] == 1556: # key after touch

          time += (s[1]-256) * 16
          patch = s[2]
          pitch = (s[3]-256-128-128) % 128
          vel = (s[4]-256-128-128-256)

          try:
              channel = patch_list.index(patch)
          except:
              channel = 15

          song_f.append(['key_after_touch', time, channel, pitch, vel])

      if s[0] == 1557: # channel after touch

          time += (s[1]-256) * 16
          patch = s[2]
          vel = (s[3]-256-128-128-256)

          try:
              channel = patch_list.index(patch)
          except:
              channel = 15

          song_f.append(['channel_after_touch', time, channel, vel])

      if s[0] == 1558: # pitch wheel change

          time += (s[1]-256) * 16
          patch = s[2]
          pitch_wheel = (s[3]-(256+128+128+256+128+16+128)) * 128

          try:
              channel = patch_list.index(patch)
          except:
              channel = 15

          song_f.append(['pitch_wheel_change', time, channel, pitch_wheel])

  detailed_stats = TMIDIX.Tegridy_SONG_to_Full_MIDI_Converter(song_f,
                                                              output_signature = 'Full MIDI Music Transformer',
                                                              output_file_name = '/content/Full-MIDI-Music-Transformer-Seed-Composition',
                                                              track_name='Project Los Angeles'
                                                              )


  #=======================================================

  print('=' * 70)
  print('Displaying resulting composition...')
  print('=' * 70)

  fname = '/content/Full-MIDI-Music-Transformer-Seed-Composition'

  x = []
  y =[]
  c = []

  colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver', 'red', 'yellow', 'green', 'cyan']

  for s in song_f:
    if s[0] == 'note':
      x.append(s[1] / 1000)
      y.append(s[4])
      c.append(colors[s[3]])

  if render_MIDI_to_audio:
    FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
    display(Audio(str(fname + '.wav'), rate=16000))

  plt.figure(figsize=(14,5))
  ax=plt.axes(title=fname)
  ax.set_facecolor('black')

  plt.scatter(x,y, c=c)
  plt.xlabel("Time")
  plt.ylabel("Pitch")
  plt.show()

else:
  print('=' * 70)

# (CONTINUATION)

In [ ]:
#@title Standard Continuation Generator

#@markdown Generation settings

number_of_prime_tokens = 600 # @param {type:"slider", min:50, max:10000, step:50}
number_of_tokens_to_generate = 600 # @param {type:"slider", min:30, max:4095, step:5}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}
temperature = 0.9 #@param {type:"slider", min:0.1, max:1, step:0.1}

#@markdown Outro generation option

try_to_generate_outro = False #@param {type:"boolean"}

#@markdown Other settings
include_prime_tokens_in_generated_output = True #@param {type:"boolean"}
allow_model_to_stop_generation_if_needed = False #@param {type:"boolean"}
render_MIDI_to_audio = True # @param {type:"boolean"}

print('=' * 70)
print('Full MIDI Music Transformer Standard Continuation Model Generator')
print('=' * 70)

if allow_model_to_stop_generation_if_needed:
  stop_token = 1561
else:
  stop_token = None

outy = melody_chords_f[:number_of_prime_tokens]

if try_to_generate_outro:
  outy.extend([1560, 1560, 1560, 1560, 1560])

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

with ctx:
  out = model.module.generate(inp,
                        number_of_tokens_to_generate,
                        temperature=temperature,
                        return_prime=include_prime_tokens_in_generated_output,
                        eos_token=stop_token,
                        verbose=True)

out0 = out.tolist()

print('=' * 70)
print('Done!')
print('=' * 70)

#======================================================================

print('Rendering results...')

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:10])
  print('=' * 70)

  if len(out1) != 0:

      song = out1
      song_f = []

      time = 0
      dur = 0
      vel = 90
      pitch = 0
      channel = 0

      son = []
      song1 = []
      for j in range(0, len(song), 5): # creating penta seqs...
          song1.append(song[j:j+5])

      patch_list = [0] * 16
      patch_list[9] = 128

      channels_list = [0] * 16
      channels_list[9] = 1

      for s in song1: # decoding...

          # 1553 - pad token

          # 1554 - patch change token
          # 1555 - control change token
          # 1556 - key after touch token
          # 1557 - channel after touch token
          # 1558 - pitch wheel change token
          # 1559 - counters seq token

          # 1560 - outro token
          # 1561 - end token
          # 1562 - start token

          if s[0] < 256: # Note

              patch = s[0]
              time += (s[1]-256) * 16
              dur = (s[2]-256-128) * 32
              pitch = (s[3]-256-128-128) % 128
              vel = (s[4]-256-128-128-256)

              if patch in patch_list:
                  channel = patch_list.index(patch)
                  channels_list[channel] = 1

              else:
                  if 0 in channels_list:
                    channel = channels_list.index(0)
                    channels_list[channel] = 1
                    song_f.append(['patch_change', time, channel, patch])

                  else:
                    channel = 15
                    channels_list[channel] = 1
                    song_f.append(['patch_change', time, channel, patch])

              song_f.append(['note', time, dur, channel, pitch, vel])

          if s[0] == 1554: # patch change

              time += (s[1]-256) * 16
              channel = (s[2]-(256+128+128+256+128))
              patch = s[3]

              if channel != 9:
                  patch_list[channel] = patch
              else:
                  patch_list[channel] = patch + 128

              song_f.append(['patch_change', time, channel, patch])

          if s[0] == 1555: # control change

              time += (s[1]-256) * 16
              patch = s[2]
              controller = (s[3]-(256+128+128+256+128+16))
              controller_value = (s[4]-(256+128+128+256+128+16+128))

              try:
                  channel = patch_list.index(patch)
              except:
                  channel = 15

              song_f.append(['control_change', time, channel, controller, controller_value])

          if s[0] == 1556: # key after touch

              time += (s[1]-256) * 16
              patch = s[2]
              pitch = (s[3]-256-128-128) % 128
              vel = (s[4]-256-128-128-256)

              try:
                  channel = patch_list.index(patch)
              except:
                  channel = 15

              song_f.append(['key_after_touch', time, channel, pitch, vel])

          if s[0] == 1557: # channel after touch

              time += (s[1]-256) * 16
              patch = s[2]
              vel = (s[3]-256-128-128-256)

              try:
                  channel = patch_list.index(patch)
              except:
                  channel = 15

              song_f.append(['channel_after_touch', time, channel, vel])

          if s[0] == 1558: # pitch wheel change

              time += (s[1]-256) * 16
              patch = s[2]
              pitch_wheel = (s[3]-(256+128+128+256+128+16+128)) * 128

              try:
                  channel = patch_list.index(patch)
              except:
                  channel = 15

              song_f.append(['pitch_wheel_change', time, channel, pitch_wheel])

      detailed_stats = TMIDIX.Tegridy_SONG_to_Full_MIDI_Converter(song_f,
                                                                  output_signature = 'Full MIDI Music Transformer',
                                                                  output_file_name = '/content/Full-MIDI-Music-Transformer-Composition_'+str(i),
                                                                  track_name='Project Los Angeles'
                                                                  )

      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Full-MIDI-Music-Transformer-Composition_'+str(i)

      x = []
      y =[]
      c = []

      colors = ['red', 'yellow', 'green', 'cyan', 'blue', 'pink', 'orange', 'purple', 'gray', 'white', 'gold', 'silver', 'purple', 'gray', 'white', 'gold', 'silver', 'red', 'yellow', 'green', 'cyan']

      for s in song_f:
        if s[0] == 'note':
          x.append(s[1] / 1000)
          y.append(s[4])
          c.append(colors[s[3]])

      if render_MIDI_to_audio:
        FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
        display(Audio(str(fname + '.wav'), rate=16000))

      plt.figure(figsize=(14,5))
      ax=plt.axes(title=fname)
      ax.set_facecolor('black')

      plt.scatter(x,y, c=c)
      plt.xlabel("Time")
      plt.ylabel("Pitch")
      plt.show()

# Congrats! You did it! :)